In [1]:
#%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import inspect
from matplotlib import pyplot as plt
from matplotlib import ticker
from scipy.constants import physical_constants
from scipy.optimize import curve_fit
from sopes import Table
from scipy.integrate import quad
from scipy.ndimage import gaussian_filter, rotate

from sopes import Table

from mpl_toolkits.mplot3d import Axes3D  

ModuleNotFoundError: No module named 'sopes'

In [ ]:
def transform(material):
    # Boltzmann constant in SESAME units kJ/K
    kB = physical_constants['Boltzmann constant'][0] / 1000. #turning this into kiloJoules because SESAME energy data is in that units
    # the lowercase k is standard for constant 


    ###############################################################################
    # Getting the SESAME data
    ###############################################################################

    # Set up the P-T table
    P_of_RT = Table('EOS_Pt_DT', material) #pressure table as a function of RT, the R stands for rho (density)
    e_of_RT = Table('EOS_Ut_DT', material) #internal energy (U) table as a function of RT
    P_of_Re = Table('EOS_Pt_DUt', material) #Pressure table as a function of R (density) and e(internal energy)
    ecold_of_R = Table('EOS_Uc_D', material) #Loads the cold curve as a function of density e_cold(rho)

    # Get the density-temperature grid from SESAME tables
    nR, nT = [int(val) for val in P_of_RT.info_many(['NR', 'NT'])] #getting the nR (number of density points) and nT
    T_grid = np.array(P_of_RT.info_one('T_Array')) #retrives actual values
    density_grid = np.array(P_of_RT.info_one('R_Array'))
    density_points, temp_points = np.meshgrid(density_grid, T_grid, indexing='ij') #each are nR x nT

    # Get the values on the grid
    P_on_grid, _, _ = P_of_RT.interpolate_many(density_points.flatten(),
                                                        temp_points.flatten()) #the _, _ just means ignore the other 2 return values
    P_on_grid = P_on_grid.reshape(nR, nT)
    e_on_grid, dedr_T, dedT_R = e_of_RT.interpolate_many(density_points.flatten(),
                                                         temp_points.flatten()) #finds pressure at every coordinate
    e_on_grid = e_on_grid.reshape(nR, nT) 
    Cv_on_grid = dedT_R.reshape(nR, nT)

    # Get the cold curve. Note that we're interpolating on nR x nT points here.
    # Also note that the temperature points are irrelevant
    ecold_on_grid, _, _ = ecold_of_R.interpolate_many(density_points.flatten(),
                                                      temp_points.flatten()) #does the same lookups for energy, interpolates using 4 points
    ecold_on_grid = ecold_on_grid.reshape(nR, nT)
    

    # Your data
    T = np.array(T_grid)
    rho = np.array(density_grid)
    E = np.array(e_on_grid)
    T_mesh, rho_mesh = np.meshgrid(T, rho)

    X = T_mesh.flatten()
    Z = rho_mesh.flatten()

    fig = plt.figure(figsize=(16, 12))

    # ---- Plot 1: Original Data ----
    Y1 = E.flatten()
    ax1 = fig.add_subplot(2, 2, 1, projection='3d')
    ax1.scatter(np.log10(X), np.log10(Y1), np.log10(Z), c=Z, cmap='viridis')
    ax1.set_xlabel('log10(Temperature)')
    ax1.set_ylabel('log10(Energy)')
    ax1.set_zlabel('log10(Density)')
    ax1.view_init(elev=30, azim=160)
    ax1.set_title(f"{material}: Original Data")

    # ---- Plot 2: Cold Curve Subtracted ----
    e_minus_cold_on_grid = e_on_grid - ecold_on_grid
    Y2 = e_minus_cold_on_grid.flatten()
    ax2 = fig.add_subplot(2, 2, 2, projection='3d')
    ax2.scatter(np.log10(X), np.log10(Y2), np.log10(Z), c=Z, cmap='viridis')
    ax2.set_xlabel('log10(Temperature)')
    ax2.set_ylabel('log10(Energy)')
    ax2.set_zlabel('log10(Density)')
    ax2.view_init(elev=30, azim=160)
    ax2.set_title(f"{material}: Cold Curve Subtracted")

    # ---- Plot 3: Divide by Cv ----
    min_Cv = 1.0e-08  # MJ / K / kg
    e_transform = e_minus_cold_on_grid / np.maximum(Cv_on_grid, min_Cv) 
    Y3 = e_transform.flatten()
    ax3 = fig.add_subplot(2, 2, 3, projection='3d')
    ax3.scatter(np.log10(X), np.log10(Y3), np.log10(Z), c=Z, cmap='viridis')
    ax3.set_xlabel('log10(Temperature)')
    ax3.set_ylabel('log10(Energy)')
    ax3.set_zlabel('log10(Density)')
    ax3.view_init(elev=30, azim=160)
    ax3.set_title(f"{material}: Cold Curve Subtracted / Cv")

    #---- Plot 4: Divide by T^α ----
    alpha = 3
    T_expanded = np.tile(T, (rho.size, 1)) 
    e_T_scaled = e_transform / np.power(T_expanded, alpha)
    Y4 = e_T_scaled.flatten()
    ax4 = fig.add_subplot(2, 2, 4, projection='3d')
    ax4.scatter(np.log10(X), np.log10(Y4), np.log10(Z), c=Z, cmap='viridis')
    ax4.set_xlabel('log10(Temperature)')
    ax4.set_ylabel('log10(Energy / T^α)')
    ax4.set_zlabel('log10(Density)')
    ax4.view_init(elev=30, azim=160)
    ax4.set_title(f"{material}: Cold Curve Subtracted/(Cv*/T^{alpha})")

    plt.tight_layout()
    plt.show()


In [ ]:
transform(3337)

In [ ]:
transform(4272)

In [ ]:
transform(2140)

In [ ]:
transform(5267)

In [ ]:
transform(5030)

## Try animating

In [ ]:
from manim import *

import numpy as np

class DataTransformation3D(ThreeDScene):
    def construct(self):
        # Set up axes
        axes = ThreeDAxes(
            x_range=[0, 10, 1],  # Adjust as per T
            y_range=[0, 10, 1],  # Adjust as per E
            z_range=[0, 10, 1],  # Adjust as per rho
            x_length=6,
            y_length=6,
            z_length=6
        )
        self.set_camera_orientation(phi=75 * DEGREES, theta=45 * DEGREES)
        self.add(axes)

        # --- STEP 1: Initial grid data (T, rho, E) ---
        # Replace with your actual data
        T = np.linspace(0, 9, 10)
        rho = np.linspace(0, 9, 10)
        E = np.random.rand(len(rho), len(T)) * 9  # Fake E just for placeholder

        # Visualize the grid points
        grid_dots = self.get_grid_dots(T, rho, E)
        self.play(*[FadeIn(dot) for dot in grid_dots])
        self.wait(1)

        # --- STEP 2: Meshgrid stage ---
        # Animate to meshgrid representation
        self.play(*[dot.animate.move_to(new_dot.get_center()) 
                    for dot, new_dot in zip(grid_dots, self.get_meshgrid_dots(T, rho, E))])
        self.wait(1)

        # --- STEP 3: Flattening ---
        # Animate to final (X, Y, Z) point cloud
        self.play(*[dot.animate.move_to(new_dot.get_center()) 
                    for dot, new_dot in zip(grid_dots, self.get_flattened_dots(T, rho, E))])
        self.wait(2)

    def get_grid_dots(self, T, rho, E):
        # Return a list of 3D Dots for original data grid (T vs rho, E as color or size)
        dots = []
        for i, r in enumerate(rho):
            for j, t in enumerate(T):
                e = E[i, j]
                dot = Dot3D(point=[t, 0, r], radius=0.05)#.set_color(blue)
                dots.append(dot)
        return dots

    def get_meshgrid_dots(self, T, rho, E):
        # Return list of 3D Dots for meshgrid: (T_mesh, rho_mesh, E)
        T_mesh, rho_mesh = np.meshgrid(T, rho)
        dots = []
        for t, r, e in zip(T_mesh.flatten(), rho_mesh.flatten(), E.flatten()):
            dot = Dot3D(point=[t, e, r], radius=0.05).set_color(GREEN)
            dots.append(dot)
        return dots

    def get_flattened_dots(self, T, rho, E):
        # Flattened view: X = T_mesh.flatten(), Y = E.flatten(), Z = rho_mesh.flatten()
        T_mesh, rho_mesh = np.meshgrid(T, rho)
        X = T_mesh.flatten()
        Z = rho_mesh.flatten()
        Y = E.flatten()
        dots = []
        for x, y, z in zip(X, Y, Z):
            dot = Dot3D(point=[x, y, z], radius=0.05).set_color(RED)
            dots.append(dot)
        return dots


In [ ]:
from manim import *

scene = DataTransformation3D()
scene.render()
